In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numba

**Exercise 2: Comparison to analytical solution**

The problem of the diffusion of a sinusoidal topography possesses an analytical solution:

$h(x,t)=h_0e^{-t/\tau_n}\sin(2n\pi x/L)$ with  for  and 
 
that corresponds to a sinusoidal function of same period as the original one but of amplitude decreasing exponentially at a rate given by the time scale ￼. This time scale is proportional to the square of the wavelength of the sinusoidal function, ￼, and inversely proportional to the diffusivity, ￼.
Compare your solution to the analytical solution and determine how the error varies as a function of ￼ and ￼.


1. Let's build a function taking as parameters $n_x$, $n_{step}$ and $n_{period}$ and returning the error (here let's define it as the maximum value of the absolute value of the difference between numerical and analytical solutions) and the condition for stability ($\Delta x^2/2K$).

In [ ]:
@numba.njit
def find_error(nx,nstep,nperiod):
    xl = 1
    x = np.linspace(0,xl,nx)
    h = np.sin(2*np.pi*x/xl*nperiod)
    dx = xl/(nx-1)
    tf = 0.005
    dt = tf/nstep
    kd = 1
    fact = kd*dt/dx**2
    tau = (xl/nperiod)**2/kd/(2*np.pi)**2
    hist = np.zeros(nstep)
    hana = np.zeros_like(hist)
    for istep in range(nstep):
        h[1:nx-1] = h[1:nx-1] + fact*(h[0:nx-2] - 2*h[1:nx-1] + h[2:nx])
        hist[istep] = h.max()
        hana[istep] = np.exp(-((istep+1)*dt/tau))
        if hist[istep]>hana[istep]*2: return np.nan, fact
    return np.abs(hana-hist).max(), fact

2. Let's now check it

In [ ]:
nx =101
nstep = 201
nperiod = 2
print(nx, nstep, nperiod, find_error(nx, nstep, nperiod))

Let's build 2 arrays containing various values of $n_x$ and $n_{step}$ and compute the error and stability condition for all of them, and then plot them

In [ ]:
nx_range = [21, 101, 201, 1001, 2001, 10001]
nstep_range = [10, 100, 1000, 10000, 100000, 1000000]
error = np.zeros((len(nx_range),len(nstep_range),2))
for i in range(len(nx_range)):
    for j in range(len(nstep_range)):
        error[i, j, :] = find_error(nx_range[i], nstep_range[j], 4)

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize=(20,6))

contour0 = ax[0].contourf(np.log10(nx_range), np.log10(nstep_range), np.log10(error[:,:,0]))
fig.colorbar(contour0, ax=ax[0])

contour1 = ax[1].contourf(np.log10(nx_range), np.log10(nstep_range), np.log10(error[:,:,1]))
fig.colorbar(contour1, ax=ax[1])

ax[0].set_xlabel('$\log_{10}(n_x)$')
ax[0].set_ylabel('$\log_{10}(n_t)$')
ax[0].set_title('$\log_{10}(error)$');

ax[1].set_xlabel('$\log_{10}(n_x)$')
ax[1].set_ylabel('$\log_{10}(n_t)$')
ax[1].set_title('$\log_{10}(condition)$');
